**Course**: Data Visualization (Prof. Dr. Heike Leitte, Jan-Tobias Sohns, TU Kaiserslautern),   **Name**: Arne Stoll,   **Date**: 27.10.2020

# Comments regarding assignments

Each assignment consists of **two pieces**:
1. A jupyter notebook with practical exercises.
2. An OLAT questionaire that contains questions regarding the material of the lecture and the notebook. 

Modalities for credit points:
- To qualify for the exam (Prüfungsvoraussetzung), you have to obtain 80% of points in each assignment.
- Points are only given through the questionaire in OLAT. Many questions will be related to material you learned or practiced in the notebook.
- While questionaires are open, you can retake them until you have enough credit points to pass.

**Submission instructions**:
- Finish the practical exercises in the notebook.
- Fill in the OLAT questionaire (which includes the submission of an HTML export of the notebook)
- The submission of the notebook is mandatory
- No group work allowed. You may discuss strategies and solutions, but every student has to do their own implementation.

<div class="alert alert-info">
    
# Assignment 1 - Visualizing Data
</div>

The **goals** of the first assignment are:
- Get familiar with python programming in the jupyter notebook;
- Be able to create a data visualization using bokeh;
- Recreate an existing visualization and develop an eye for key features;
- Start critical thinking about design options;



To achieve these goals, your task is to create a visualization of the weather in Kaiserslautern in 2018. The visualization should be similar to the following chart from the New York Times (Jan. 11, 1981, p. 32; Tufte (1983), p. 30) and needs to be implemented in bokeh+pandas:

![New York city's weather for 1980 from the New York Times](http://euclid.psych.yorku.ca/SCS/Gallery/images/NYweather.jpg)


<div class="alert alert-danger">

**Important**: While no points will be awarded for typing the correct answers in the notebooks, it is highly advised to solve the tasks thoroughly. They are designed to be encouraging and provide you with valuable learnings for the exam, understanding of the methods and practical coding.
</div>

<div class="alert alert-success">
    
All tasks in this notebook are marked in green.
</div>

<div class="alert alert-info">
    
## 1. Starter Code - Minimal working example
</div>

The following pieces of code load the data for this assignment and generate a minimal chart for the temperature data. More details can be found in the [bokeh documentation](https://docs.bokeh.org/en/latest/docs/user_guide/quickstart.html).

First load all necessary python modules:

In [ ]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Band, ColumnDataSource, PrintfTickFormatter, DatetimeTickFormatter, Label, HoverTool
from bokeh.layouts import column
from bokeh.models.tickers import MonthsTicker

output_notebook()

Load the data given in csv-file format using the pandas library and display the first lines of the data table.

In [ ]:
df_kl = pd.read_csv('KLweather2018.csv', parse_dates=['Timestamp'], index_col='Timestamp')
df_kl_prec = pd.read_csv('KLweather2018_monthlyPrecipitation.csv', parse_dates=['Timestamp'], index_col='Timestamp')
        
df_kl_prec.head()

Plot the temperature minimum as a line chart with bokeh using default settings. 

In [ ]:
# create a figure
p = figure(plot_height=400, x_axis_type="datetime")

# define the type of glyph that is rendered and its data. here: a polyline
p.line(source=df_kl, x='Timestamp', y='temp_min')

# render the chart
show(p)

<div class="alert alert-info">
    
## 2. Customizing the temperature chart
</div>

As detailed above, your visualization should look like a modern version of the one from the New York Times. This can be achieved by changing the graphical elements and styling visual properties. In the function below some elements are already changed. Update the code to make the temperature chart even more similar:

<div class="alert alert-success">
    
- Depict the normal high and low temperatures as polylines.
- Label the two polylines. You may use the legend functionality.
- Depict the daily temperature range as an area.
- Label the y-axis.
- Style visual attributes (color, line style) to your liking.
    
</div>

Helpful ressources:
- [Plotting with basic glyphs](https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html) - Overview of glyph types that are implemented in bokeh; see the examples for all the graphical primitives that can be plotted directly.
- [Styling visual attributes](https://docs.bokeh.org/en/latest/docs/user_guide/styling.html) - See styling options for chart elements

In [ ]:
from bokeh.models import LabelSet

def create_temperature_chart(df, width=900):
    '''Create a bokeh figure for temperature range and normal values.'''
    # create figure and data source
    p = figure(plot_width=width, plot_height=400, title='Kaiserslautern\'s Weather for 2018', tools=['xwheel_zoom'], 
           x_axis_type="datetime", x_axis_location="above", y_range=(-15,40))
    
    source = ColumnDataSource(df)
    
    # add graphical items
    p.varea(source=source, x='Timestamp', y1='temp_min', y2='temp_max', fill_alpha=0.5, color='black')
    p.line(source=source, x='Timestamp', y='temp_normal_max', alpha=0.8, line_color="red", legend="normal high")
    p.line(source=source, x='Timestamp', y='temp_normal_min', alpha=0.8, line_color="black", legend="normal low")
    p.legend.location = "top_left"
    # mark min/max temperature
    tmax_id = df_kl['temp_max'].idxmax()
    tmin_id = df_kl['temp_min'].idxmin()
    v1 = df_kl.at[tmax_id,'temp_max']
    v2 = df_kl.at[tmax_id,'temp_min']
    
    data = ColumnDataSource(dict(
        dates=[tmax_id, tmin_id],
        temps=[df_kl.at[tmax_id,'temp_max'], df_kl.at[tmin_id,'temp_min']],
        names=[('High ' + str(v1) + ' °C'), ('Low ' + str(v2)) + ' °C']))
    
    p.scatter(source=data, x='dates', y='temps', color='black')
    labels = LabelSet(x='dates', y='temps', text='names', x_offset=2, y_offset=2, source=data, 
                      background_fill_color='grey', background_fill_alpha=0.5, text_font_size='10pt')
    p.add_layout(labels)
    # style visual attributes
    p.xaxis.ticker = MonthsTicker(months=list(range(12))) 
    p.xgrid.ticker = MonthsTicker(months=list(range(12))) 
    p.xaxis.formatter=DatetimeTickFormatter(months=["           %b"])
    p.xaxis.major_label_text_align = 'right'
    p.yaxis[0].formatter = PrintfTickFormatter(format="%2i°")
    p.yaxis.axis_label = "Temperature [°C]"
    p.title.text_font_size = "15pt"
    p.title.align = "center"
    
    return p

p = create_temperature_chart(df_kl)
show(p)

<div class="alert alert-info">
    
## 3. Filtering data and making annotations
</div>

The following piece of code demonstrates how to find maxima in a data column. Use this code to automatically find the highest and lowest temperature values in 2018 and place a mark in the chart above at these positions (e.g. circle the respective data points).

<div class="alert alert-success">
    
- Automatically filter the highest and lowest temperatures in Kaiserslautern in 2018.
- Integrate the code in the chart computation method above and mark the two detected positions.
- Add text labels to the positions. [Label documentation](https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html#labels) for bokeh.
    
</div>

In [ ]:
tmax_id = df_kl['temp_max'].idxmax()
tmin_id = df_kl['temp_min'].idxmin()

print("KL temperature maximum:", tmax_id, df_kl.at[tmax_id,'temp_max'])
print("KL temperature minimum:", tmin_id, df_kl.at[tmin_id,'temp_min'])

<div class="alert alert-info">

## 4. Designing additional charts
</div>

Now design the charts for precipitation and relative humidity.

<div class="alert alert-success">
    
- Create the chart for precipitation. Try to design a bar chart using the hints below.
- Create the chart for humidity.
    
</div>

Hints for temporal x-axis:
- **Width of bars**: The width is given milliseconds. In order to get the required scaling, you will need to specify the width like: `widthInDays = ndays*24*60*60*1000` (24 hours * 60 minutes * 60 seconds * 1000 milliseconds)
- **Position of bars**: You can shift the bars using the dodge function `x=dodge('prec', value, range=p.x_range)`. Keep in mind that you need to define an appropriate `value` by which to shift the bar.

In [ ]:
def create_precipitation_chart(df, width=900):
    '''Create a bokeh figure for monthly precipitation (2018 vs normal values).'''
    day_offset = 10
    d = {'time1':[i + pd.tseries.offsets.DateOffset(days=-(day_offset/2.0)+1) for i, row in df.iterrows()],
         'time2':[i + pd.tseries.offsets.DateOffset(days=1+(day_offset/2.0)+1) for i, row in df.iterrows()]
         , 'prec':df['prec'], 'prec_normal':df['prec_normal']}
    df = pd.DataFrame(data=d)    
    p = figure(plot_width=width, plot_height=200, tools=['xwheel_zoom'], x_axis_type="datetime")    
    p.vbar(x='time1', top='prec', source= df, width=10*24*60*60*1000, color='black',
           legend='actual precipitation', alpha=.75)
    p.vbar(x='time2', top='prec_normal', source=df, width=10*24*60*60*1000, legend='normal precipitation', 
           hatch_pattern='/', hatch_scale=9, color='red', alpha=0.75, hatch_alpha=0.75)
    
    p.xgrid.grid_line_color = None
    p.legend.orientation = "vertical"
    p.legend.location = "top_center"
    
    return p

show(create_precipitation_chart(df_kl_prec))

In [ ]:
def create_humidity_chart(df, width=900):
    '''Create a bokeh figure for relative humidity.'''
    
    p = figure(plot_width=width, plot_height=200, tools=['xwheel_zoom'], x_axis_type="datetime")
    lst = [0]*len(df['rel_humidity'])
    df['zeros'] = lst
    p.varea(source=df_kl, x='Timestamp', y1='rel_humidity', y2='zeros', color='black', alpha=0.75)
    p.yaxis[0].formatter = PrintfTickFormatter(format="%2i")
    p.yaxis.axis_label = "Percentage [%]"
    p.title.text_font_size = "15pt"
    p.title.align = "center"
    return p

show(create_humidity_chart(df_kl))

<div class="alert alert-info">
    
## 5. Combining multiple charts
</div>

In this last part, we combine the three charts you designed above.

<div class="alert alert-success">
    
- Create the combined weather chart for Kaiserslautern.
- Save a jpg/png-version or screenshot of this chart that can be uploaded in OLAT.
    
</div>

In [ ]:
show(column(create_temperature_chart(df_kl), create_precipitation_chart(df_kl_prec),create_humidity_chart(df_kl)))